# <span style="color: #FF0000">Exemple de machine de turing</span>

In [1]:
from ipywidgets import Button, Textarea, Layout,Label,HBox,HTML,Dropdown
from IPython.display import display,clear_output
import time
import threading

### Ajouter 1 à un binaire

Une machine de Turing comporte un nombre d'états finis (ici 3 états e1, e2 et e3)
Lorsque la machine est dans un état, elle va avoir un comportement pour chaque valeur possible du bit lu sur le ruban 0, 1 ou rien. Par exemple ici, dans l'état e1:  
=> la lecture d'un 0 va conduire la machine à écrire un 0 et à passer à l'état e2 en se déplaçant vers la droite du ruban.  
=> la lecture d'un 1 va conduire la machine à écrire un 1 et à passer à l'état e2 en se déplaçant vers la droite du ruban.  
=> la lecture d'une case vide va conduire la machine à ne rien écrire et à rester dans l'état e1 en se déplaçant vers la droite du ruban.  
Ainsi dans l'état e1, la machine ne change pas l'information et se déplace à tous les coups vers la droite. Elle change d'état lorsqu'elle rencontre la première case non vide. L'état e1 cherche donc le début du binaire sur le ruban.  
  
De même on peut voir que l'état e2 cherche la fin du binaire et se met dans l'état e3 sur le bit le plus à droite du binaire une fois celui-ci trouvé.  
  
Dans l'état e3, la machine va modifier les valeurs sur le ruban jusqu'à obtenir le binaire de départ plus 1


In [2]:
Etat = {}
e1= {}
e1["0"] = ("e1","0","0","droite","e2")
e1["1"] = ("e1","1","1","droite","e2")
e1[" "] = ("e1"," "," ","droite","e1")
e2 = {}
e2["0"] = ("e2","0","0","droite","e2")
e2["1"] = ("e2","1","1","droite","e2")
e2[" "] = ("e2"," "," ","gauche","e3")
e3 = {}
e3["0"] = ("e3","0","1","gauche","fin")
e3["1"] = ("e3","1","0","gauche","e3")
e3[" "] = ("e3"," ","1","droite","fin")

Etat["fin"]=None
Etat["e1"]=e1
Etat["e2"]=e2
Etat["e3"]=e3

liste = []
for k, v in Etat.items():
    if k!="fin":
        for k2, v2 in v.items():
            b = "|".join(v2)
            b=b.replace(" ","VIDE")
            liste.append("|"+b+"|")
a = "|Etat|Lit|Ecrit|Déplace|Suivant|\n|---|---|---|---|---|\n"+"\n".join(liste)

{{print(a)}}

In [3]:
def on_value_change(change=None,i=None):
    global Ruban,dd,Rubani
    Rubani[i]=dd[i].value
    
def arret(b):
    global auto,stop,raz,lock
    stop.disabled=True
    raz.disabled=True
    auto=False
    while lock:
        pass
    stop.disabled=False
    raz.disabled=False

def debut(b):
    global go,Rubani,eiw,dd,step,e,c,auto,lock,stop,raz,premier_pas
    stop.disabled=True
    raz.disabled=True
    auto=False
    premier_pas = True
    while lock:
        pass
    stop.disabled=False
    raz.disabled=False
    Rubani=[" " for i in range(10)]
    Rubani[1]="1"
    Rubani[2]="0"
    Rubani[3]="1"
    for i in range(10):
        dd[i].value=Rubani[i]    
    clear_output()
    display(eiw)
    display(HBox([go,step,stop,raz]))
    for el in dd:
        el.disabled=False
    dd[0].disabled=True
    dd[-1].disabled=True
    e="e1"
    c=0
    
def on_button_click(b):
    global auto, premier_pas
    auto=True
    if premier_pas:
        suite()
        premier_pas = False
    else:
        suite2()

def work():
    global go,Ruban,Rubani,eiw,dd,step,e,c,auto,stop,raz, premier_pas
    turing()
    while auto and e!="fin":
        turing()
    go.disabled=False
    step.disabled=False
    if e == "fin":
        for el in dd:
            el.disabled=False
        dd[0].disabled=True
        dd[-1].disabled=True
        e = "e1"
        c = 0
        premier_pas = True
    return

def unpas(b):
    global auto, premier_pas
    auto=False
    if premier_pas:
        suite()
        premier_pas = False
    else:
        suite2()
    
def suite():
    global go,Ruban,Rubani,eiw,dd,step,e,c,auto,stop,raz
    clear_output()
    display(eiw)
    display(HBox([go,step,stop,raz]))
    t = "".join(Rubani)
    t = t.strip()
    t = t.split(" ")
    if len(t)>1:
        return
    fin = True
    for el in Rubani:
        if el!=" ":
            fin = False
            break
    if fin:
        return
    for el in dd:
        el.disabled=True
    go.disabled=True
    step.disabled=True
    Ruban=Rubani[:]
    affichage()
    thread = threading.Thread(target=work)
    thread.start()
    
def suite2():
    global go,Ruban,Rubani,eiw,dd,step,e,c,auto,stop,raz
    for el in dd:
        el.disabled=True
    go.disabled=True
    step.disabled=True
    thread = threading.Thread(target=work)
    thread.start()
    
    
def etatinit():
    global Ruban,dd,go,Rubani,titre,eiw,step,e,c,lock,stop,raz, premier_pas
    premier_pas = True
    lock = False
    Rubani=[" " for i in range(10)]
    Rubani[1]="1"
    Rubani[2]="0"
    Rubani[3]="1"
    e="e1"
    c=0
    dd = []
    for i in range(10):
        dd.append(Dropdown(
    options=[' ', '0', '1'],
    value=Rubani[i],
    description='',
    disabled=False,
    layout=Layout(width='48px',height='48px'),
))
    dd[0].disabled=True
    dd[-1].disabled=True
    
    titre=HTML(value = f"<center><b><font color='black'>Ruban Initial</b></center>", layout = Layout(width='100px',height='30px'))
    
    for i in range(10):
        dd[i].observe(lambda change,x=i:on_value_change(change,x),names="value")
    go = Button(description="GO",disabled=False,layout=Layout(width="100px",height="30px"))
    step = Button(description = "1 STEP",disabled=False,layout=Layout(width="100px",height="30px"))
    stop = Button(description = "STOP",disabled=False,layout=Layout(width="100px",height="30px"))
    raz = Button(description = "RESET",disabled=False,layout=Layout(width="100px",height="30px"))
    eiw=HBox([titre]+dd)
    #display(titre)
    display(eiw)
    display(HBox([go,step,stop,raz]))
    go.on_click(on_button_click)
    step.on_click(unpas)
    stop.on_click(arret)
    raz.on_click(debut)

            
def affichage():
    global Etat,Ruban,wcases,wetat,etatlab,entete,wentete,wetat,wcases,wruban
    titre2=HTML(value = f"<center><b><font color='black'>Ruban</b></center>", layout = Layout(width='100px',height='30px'))
    #display(titre2)
    
    entete = []
    enteteval =[" ","lit","ecrit","deplace","suivant"]
    for i in range(5):
        entete.append(HTML(value = f"<center><b><font color='black'>"+enteteval[i]+"</b></center>", layout = Layout(width='100px',height='30px')))
    wentete = HBox(entete)
    display(wentete)                  
    etatval = ["e1","1","1","droite","e1"]
    etatlab = []
    for i in range(5):
        etatlab.append(HTML(value = f"<center><b><font color='black'>"+etatval[i]+"</b></center>", layout = Layout(width='100px',height='30px')))
    wetat = HBox(etatlab)
    display(wetat)
    wcases = []
    for i in range(len(Ruban)):
        wcases.append(HTML(value = f"<center><b><font color='black'>"+Ruban[i]+"</b></center>", layout = Layout(width='48px',height='48px',border='3px solid black',)))
    wruban = HBox([titre2]+wcases)
    display(wruban)

def turing():
    global Etat,Ruban,wcases,wetat,etatlab,lock,e,c
    lock=True
    #etat = Etat[e][Ruban[case]]
    etatval = Etat[e][Ruban[c]]
    for i in range(5):
        etatlab[i].value = f"<center><b><font color='black'>"+etatval[i]+"</b></center>"
    for i in range(len(wcases)):
        wcases[i].layout.border = '3px solid black'

    wcases[c].layout.border = '3px solid red'
    
    for i in range(1,5):
        etatlab[i-1].layout.border = '0px'
        etatlab[i].layout.border = '3px solid red'
        time.sleep(0.25)
        if i==2:
            Ruban[c]=etatval[2]
            wcases[c].value = f"<center><b><font color='black'>"+Ruban[c]+"</b></center>"
        elif i == 3:
            wcases[c].layout.border = '3px solid black'
            if etatval[3]=="droite":
                c+=1
            else:
                c-=1
            wcases[c].layout.border = '3px solid red'
        elif i==4:
            e = etatval[4]
        time.sleep(0.5)
    etatlab[-1].layout.border = '0px'
    if e == "fin":
        wcases[c].layout.border = '3px solid black'
    lock=False
    return
  

etatinit()

['1010', '0']
